## Context:



To Identify Sentiment Analysis of give input is number(may be Encrypted or already Vectorized ),so cannot we implement Lemmitization or Vectorization ,Now i'm going to build a model with Deep Learning Technique of LSTM

## Importing Library

In [39]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import one_hot

import warnings 
warnings.filterwarnings("ignore")



## Reading Data

In [41]:
df_train = pd.read_csv("/content/sample_data/training.csv")
df_test = pd.read_csv("/content/sample_data/test_.csv")

## Deep Learning Technique of LSTM:


### Tokenization

In [42]:
corpus_train = []
for sent in df_train["text"]:
    review = sent.split()
    review = " ".join(review)
    corpus_train.append(review)

In [43]:
corpus_test = []
for sent in df_test["text"]:
    review = sent.split()
    review = " ".join(review)
    corpus_test.append(review)

### One Hot Representation

In [44]:
# To set the Vocab size
sent_tokenize = []

for sent in df_train["text"]:
    sentence = sent.split()
    sent_tokenize.append(sentence)
lis1 = []
for sent in sent_tokenize:
    for word in sent:
        lis1.append(word)

#### Unique Word Identiffication function we can laso use set function

In [45]:
### This one can find using set() function but i use scratch code 
### This code is not necessary
def unique_vector(com_vector):
    
    unique = []
    

    for vec in com_vector:
        
        l = len(unique)
        
        if vec =="":
            break
        elif l==0:
            unique.append(vec)
        else:
            
            if vec in unique:
                pass
            else:
                unique.append(vec)
    return unique


In [46]:
uni = unique_vector(lis1)
len(uni)

8493

In [47]:
#voc size
#from the analysis the unique feature is 8493
#so i choosen 10000 as a vocab size
vocb_size = 10000 

one_hot_rep = [one_hot(words,vocb_size) for words in corpus_train]
one_hot_rep_test = [one_hot(words,vocb_size) for words in corpus_test]



### Word Embedding Representation

In [48]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
# pad_sequences is very very important bcz whenever passing anything to the embedding layer everything should be in same number of words
from tensorflow.keras.models import Sequential

### Padding

In [49]:
sent_tokenize = []
sent_length = []

for sent in df_train["text"]:
    sentence = sent.split()
    sent_len = len(sentence)
    sent_tokenize.append(sentence)
    sent_length.append(sent_len)
max(sent_length)        

6290

In [50]:
sent_tokenize = []
sent_length = []

for sent in df_test["text"]:
    sentence = sent.split()
    sent_len = len(sentence)
    sent_tokenize.append(sentence)
    sent_length.append(sent_len)
max(sent_length)        

3122

#### Setting Parameter for Padding 

In [51]:
pad_len_size = 7000
embedded_docs = pad_sequences(one_hot_rep,padding = "pre",maxlen =pad_len_size )
embedded_docs_test = pad_sequences(one_hot_rep_test,padding = "pre",maxlen =pad_len_size )

print(embedded_docs)


[[   0    0    0 ... 5226 4629 1043]
 [   0    0    0 ... 4381  814   57]
 [   0    0    0 ... 1072 8648 1043]
 ...
 [   0    0    0 ... 1043 7026 2644]
 [   0    0    0 ... 9329 3922 7631]
 [   0    0    0 ... 3789 3786 9100]]


### Embedding Layer

In [52]:
embedding_vector_feature = 40
#Emmbedded Layer
model = Sequential()
model.add(Embedding(vocb_size,embedding_vector_feature,input_length =pad_len_size ))
model.add(LSTM(100))
model.add(Dense(1,activation = "sigmoid"))
model.compile(loss = 'binary_crossentropy',optimizer = "adam",metrics = ["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7000, 40)          400000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [53]:
X_train = np.array(embedded_docs)
y_train = np.array(df_train["category"])

In [54]:
print(X_train.shape)
print(y_train.shape)

(3464, 7000)
(3464,)


In [55]:
X_test = np.array(embedded_docs_test)
print(X_test.shape)

(1360, 7000)


## ReSampling

In [56]:
from imblearn.combine import SMOTETomek
from collections import Counter
os = SMOTETomek()
X_train,y_train = os.fit_sample(X_train,y_train)

print("Counter After : {} ".format(Counter(y_train)))


Counter After : Counter({0: 3348, 1: 3348}) 


## Prediction

In [38]:
model.fit(X_train,y_train,epochs = 10,batch_size = 64)

Epoch 1/10
105/105 [==============================] - 1079s 10s/step - loss: 0.2545 - accuracy: 0.9011
Epoch 2/10
105/105 [==============================] - 1067s 10s/step - loss: 0.0603 - accuracy: 0.9813
Epoch 3/10
105/105 [==============================] - 1072s 10s/step - loss: 0.0210 - accuracy: 0.9976
Epoch 4/10
105/105 [==============================] - 1074s 10s/step - loss: 0.0087 - accuracy: 0.9991
Epoch 5/10
105/105 [==============================] - 1080s 10s/step - loss: 0.0045 - accuracy: 0.9996
Epoch 6/10
105/105 [==============================] - 1055s 10s/step - loss: 0.0026 - accuracy: 0.9997
Epoch 7/10
105/105 [==============================] - 1057s 10s/step - loss: 0.0017 - accuracy: 0.9999
Epoch 8/10
105/105 [==============================] - 1062s 10s/step - loss: 0.0011 - accuracy: 0.9999
Epoch 9/10
105/105 [==============================] - 1054s 10s/step - loss: 5.5744e-04 - accuracy: 1.0000
Epoch 10/10
105/105 [==============================] - 1046s 10s/step

### Performance Metrics and Accuracy Score

In [58]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [59]:
pred_test = model.predict_classes(X_test)
pred_test

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int32)

In [60]:
df_test = pd.read_csv("/content/sample_data/test_.csv")

In [61]:
pred_df = pd.DataFrame(data = df_test["id"])
pred_df["category"] = pred_test
pred_df

,id,category
0,3729,0
1,3732,0
2,3761,0
3,5,0
4,7,0
...,...,...
1355,311,0
1356,324,1
1357,329,0
1358,342,0


In [62]:
pred_df.to_csv(r'/content/sample_data/Result/Predicted_Result_LSTM.csv',index = False)

In [63]:
result_df = pd.read_csv("/content/sample_data/Result/Predicted_Result_LSTM.csv")

In [64]:
result_df

,id,category
0,3729,0
1,3732,0
2,3761,0
3,5,0
4,7,0
...,...,...
1355,311,0
1356,324,1
1357,329,0
1358,342,0


In [65]:
result_df["category"].value_counts()

0    978
1    382
Name: category, dtype: int64

Here i Build a model with use Deep Learning Technique of LSTM ,Comparing with other models and Techniques of Bag of Words with Naive Bayes and TF- IDF with Naive Bayes,This Model is Optimized and Performed well.